In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import PyPDF2

Descargar recursos de NLTK

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

Extracción de texto de los PDFs y cargarlos en un dataframe

In [ ]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_directory = 'pdf/'

documents = []
for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)
        text = extract_text_from_pdf(pdf_path)
        documents.append(text)
        
df = pd.DataFrame(documents, columns=['text'])

Tokenización de palabras

In [ ]:
df['tokens'] = df['text'].apply(word_tokenize)

Eliminación de stopwords

In [ ]:
stop_words = set(stopwords.words('english'))
df['filtered_tokens'] = df['tokens'].apply(lambda x: [word for word in x if word.lower() not in stop_words])


Concatenar todas las palabras de todos los documentos para análisis de frecuencia

In [ ]:
all_words = [word.lower() for tokens in df['filtered_tokens'] for word in tokens]
word_freq = Counter(all_words)

# Estadísticas Descriptivas

## Frecuencia de palabras

In [ ]:

print("Frecuencia de palabras:")
print(word_freq.most_common(10))

# Longitud de documentos

In [ ]:
df['doc_length'] = df['filtered_tokens'].apply(len)
print("\nLongitud promedio de documentos:", df['doc_length'].mean())
print("Distribución de la cantidad de palabras por documento:")
print(df['doc_length'].describe())

Gráfica de la distribución de la longitud de los documentos

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['doc_length'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribución de la cantidad de palabras por documento')
plt.xlabel('Cantidad de palabras')
plt.ylabel('Frecuencia')
plt.show()

## Nube de Palabras

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Nube de Palabras')
plt.axis('off')
plt.show()

## Distribución de Términos

In [ ]:
common_words = [word for word, freq in word_freq.most_common(10)]

plt.figure(figsize=(10, 6))
plt.bar(common_words, [word_freq[word] for word in common_words], color='lightgreen')
plt.title('Frecuencia de Palabras Comunes')
plt.xlabel('Palabras')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.show()